In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys

In [4]:
pred =pd.read_csv('../data/hungarian.csv')

In [5]:
pred = pred.drop(['Unnamed: 0'],axis ='columns')

In [6]:
tn = np.array(pred['취급액'])
tn.reshape((1,-1))
final  = pd.DataFrame(tn.reshape((1,-1)).reshape((60,-1)).T,columns=pred['방송일시'].unique(),index=pred['상품코드'].unique()).T
final = final.astype(int)

In [8]:
tmp = np.zeros((20,final.shape[1]))
final_tmp = np.array(final)
for i in tqdm(range(final_tmp.shape[1])):
  sum = 0
  for j in range(60):
    if(j%3==2):
        tmp[j//3][i] = sum + final_tmp[j][i]
        sum = 0
    else:
      sum+=final_tmp[j][i]


100%|██████████| 389/389 [00:00<00:00, 17378.17it/s]


In [10]:
ffinal = pd.DataFrame(tmp)
ffinal.columns = final.columns

In [12]:
my_index =[]
for i in range(0,len(final.index),3):
  #print(final.index[i])
  my_index.append(final.index[i])

In [13]:
ffinal.index = my_index

In [14]:
col = []
for i in ffinal.columns:
    col.append(pred[pred['상품코드']==i].reset_index()['상품명'][0])

In [15]:
ffinal.columns = col

In [16]:
from munkres import Munkres, print_matrix

m = Munkres()

In [18]:
def hung2(df):

  idx = df.index
  pro = df.columns
  pro_ =[]

  matrix = df.values.tolist()

  row1 = []
  col1 = []
  weight = []

  cost_matrix = []
  for row in matrix:
      cost_row = []
      for col in row:
          cost_row += [sys.maxsize - col]
      cost_matrix += [cost_row]

  m = Munkres()
  indexes = m.compute(cost_matrix)
#   print_matrix(matrix, msg='Highest profit through this matrix:')
  total = 0
  for row, column in indexes:   # indexes : [(0, 0), (1, 7), (2, 19), ... ] 
      value = matrix[row][column]
      total += value
#       print('(%d, %d) -> %d' % (row, column, value))
      row1.append(row) 
      col1.append(column)
      weight.append(value)

  for i in range(len(col1)):
    pro_.append(pro[col1[i]])

  #print('total cost: %d' % total)
  final = pd.DataFrame(columns=[idx])   # 최종 dataframe 생성.
  final = final.T                         # 시간대를 index로 주기위함.
  final['product'] = pro_           
  final['weight'] =weight  # 상품별 해당시간에 예측된 취급액 

  return final, total


In [19]:
hungmat , total = hung2(ffinal)

In [20]:
hungmat

,product,weight
2020-06-01 06:40:00,LG 울트라HD TV AI ThinQ(인공지능 씽큐) 75형 75UN7850KNA,198858856.0
2020-06-01 07:40:00,무이자 삼성 UHDTV 65형 KU65UT7000FXKR,195402133.0
2020-06-01 08:40:00,PN 메가티타늄 후라이팬+IH 인덕션 1구 풀세트,173701742.0
2020-06-01 09:40:00,클라쎄 벽걸이 에어컨 TDOZ-S10JK,205383409.0
2020-06-01 10:40:00,3인용 우아미천연가죽소파,160186154.0
2020-06-01 11:40:00,마르엘라로사티 플로럴썸머원피스3종,229911434.0
2020-06-01 12:40:00,LG 울트라HD TV AI ThinQ(인공지능 씽큐) 75형 75UN7850KNA,139077802.0
2020-06-01 13:40:00,3D에어매쉬 편백나무베개 (패밀리),131483220.0
2020-06-01 14:40:00,2020 안동간고등어 20팩,175215977.0
2020-06-01 15:40:00,에지리 스텐큐브 프라이팬,180152995.0


In [21]:
total

3547395431.0

In [22]:
hungmat.to_excel('../편성표/20200601_pred.xlsx',index=False)